# 모듈 불러오기

#### 기본

In [1]:
import numpy as np
import pandas as pd
import joblib

In [2]:
# !conda install numpy 
# !conda install pandas
# !conda install scikit-learn
# !conda install scipy
# !conda install tensorflow
# !conda install matplotlib
# !conda install seaborn

# !pip install xgboost
# !pip install lightgbm
# !pip install catboost
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !pip install hyperopt
# !pip install -U imbalanced-learn
# !pip install missingno

#### 전처리

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance

from sklearn import impute
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer

#### 리샘플링

In [4]:
from imblearn.over_sampling import (
    RandomOverSampler, 
    ADASYN, 
    SMOTE
)
from imblearn.under_sampling import (
    RandomUnderSampler, 
    TomekLinks, 
    CondensedNearestNeighbour, 
    OneSidedSelection, 
    EditedNearestNeighbours, 
    NeighbourhoodCleaningRule
)

#### 분석

In [5]:
from scipy.stats import skew, kurtosis
from scipy.stats import ttest_ind, f_oneway, pearsonr

#### 회귀

In [6]:
from sklearn.linear_model import LinearRegression as RL
from sklearn.neighbors import KNeighborsRegressor as KNNR
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from xgboost import XGBRegressor as XGBR
from lightgbm import LGBMRegressor as LGBMR
from catboost import CatBoostRegressor as CBR

from lightgbm import plot_importance as lgbm_plot_importance
from xgboost import plot_importance as xgb_plot_importance
from catboost import Pool

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

#### 분류

In [7]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNNC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RFC
from xgboost import XGBClassifier as XGBC
from lightgbm import LGBMClassifier as LGBMC
from catboost import CatBoostClassifier as CBC

from sklearn.metrics import confusion_matrix as cmatrix
from sklearn.metrics import classification_report as creport
from sklearn.metrics import recall_score as recall
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

#### 교차검증

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from catboost import cv
import hyperopt

from sklearn.model_selection import (
    StratifiedKFold, # 분류
    KFold, # 회귀
    # GroupKFold, 
    # RepeatedKFold, 
    # StratifiedGroupKFold, 
    # RepeatedStratifiedKFold
)

#### 시각화

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')  

import warnings
warnings.filterwarnings(action='ignore')
%config InlineBackend.figure_format = 'retina'

# 데이터 로드

In [10]:
folder_name = '건물'
file_name = 'GIS건물통합정보_fix'

df_all_origin = pd.read_excel(f'./data/{folder_name}/{file_name}.xlsx', engine='openpyxl')

In [11]:
folder_name = '건물'
file_name = 'GIS건물통합정보_강서구'

df_origin = pd.read_excel(f'./data/{folder_name}/{file_name}.xlsx', engine='openpyxl')

In [12]:
df = df_origin.copy()

# 데이터 이해

In [15]:
df_all_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667455 entries, 0 to 667454
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   법정동명    667455 non-null  object 
 1   지번      667455 non-null  object 
 2   용도명     539521 non-null  object 
 3   구조명     539288 non-null  object 
 4   면적      667455 non-null  float64
 5   사용승인일자  538243 non-null  object 
 6   연면적     667455 non-null  float64
 7   대지면적    667455 non-null  float64
 8   높이      667455 non-null  float64
 9   건폐율     667455 non-null  float64
 10  용적율     667455 non-null  float64
dtypes: float64(6), object(5)
memory usage: 56.0+ MB


In [16]:
df_all_origin.isna().sum()

법정동명           0
지번             0
용도명       127934
구조명       128167
면적             0
사용승인일자    129212
연면적            0
대지면적           0
높이             0
건폐율            0
용적율            0
dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28900 entries, 0 to 28899
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   법정동명    28900 non-null  object        
 1   지번      28900 non-null  object        
 2   용도명     28900 non-null  object        
 3   구조명     28900 non-null  object        
 4   면적      28900 non-null  float64       
 5   사용승인일자  28900 non-null  datetime64[ns]
 6   연면적     28900 non-null  float64       
 7   대지면적    28900 non-null  float64       
 8   높이      28900 non-null  float64       
 9   건폐율     28900 non-null  float64       
 10  용적율     28900 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(4)
memory usage: 2.4+ MB


In [14]:
df.isna().sum()

법정동명      0
지번        0
용도명       0
구조명       0
면적        0
사용승인일자    0
연면적       0
대지면적      0
높이        0
건폐율       0
용적율       0
dtype: int64

# 동별 노후도

In [29]:
df['사용승인일자'] = df['사용승인일자'].str.slice(0, -1)

0        1900
1        1900
2        1900
3        1900
4        1900
         ... 
28895    1900
28896    1992
28897    1992
28898    1979
28899    1900
Name: 사용승인일자, Length: 28900, dtype: int64

In [23]:
df['사용승인년도'] = df['사용승인일자'].dt.year

In [24]:
# folder_name = '건물'
# file_name = 'GIS건물통합정보_강서구'
# df.to_excel(f'./data/{folder_name}/{file_name}.xlsx', engine='openpyxl')

In [25]:
group_df = df.groupby(by=['법정동명'], as_index=False)['사용승인년도'].value_counts()

In [30]:
group_df

,법정동명,사용승인년도,count
0,서울특별시 강서구 가양동,1900,333
1,서울특별시 강서구 가양동,1992,123
2,서울특별시 강서구 가양동,1993,81
3,서울특별시 강서구 가양동,1999,29
4,서울특별시 강서구 가양동,1995,25
...,...,...,...
502,서울특별시 강서구 화곡동,1968,43
503,서울특별시 강서구 화곡동,1967,20
504,서울특별시 강서구 화곡동,1966,4
505,서울특별시 강서구 화곡동,1959,1
